**В задании предлагается с помощью Pandas ответить на несколько вопросов по данным о сердечно-сосудистых заболеваниях (качать данные не надо, они уже есть в репозитории). Данные использовались в соревновании Ml Boot Camp 5.**

**Заполните код в клетках (где написано "Ваш код здесь") и ответьте на вопросы в веб-форме.**

В соревновании предлагалось определить наличие/отсутствие сердечно-сосудистых заболеваний (ССЗ) по результатам осмотра пациента.

**Описание данных.**

Датасет сформирован из реальных клинических анализов, и в нём используются признаки, которые можно разбить на 3 группы:

Объективные признаки:

* Возраст (age)
* Рост (height)
* Вес (weight)
* Пол (gender)

Результаты измерения:

* Артериальное давление верхнее и нижнее (ap_hi, ap_lo)
* Холестерин (cholesterol)
* Глюкоза (gluc)

Субъективные признаки (со слов пациентов):

* Курение (smoke)
* Употребление алкоголя (alco)
* Физическая активность (active)

Целевой признак (который интересно будет прогнозировать):

* Наличие сердечно-сосудистых заболеваний по результатам классического врачебного осмотра (cardio)

Значения показателей холестерина и глюкозы представлены одним из трех классов: норма, выше нормы, значительно выше нормы. Значения субъективных признаков — бинарны.

Все показатели даны на момент осмотра.

Мы будем работать только с обучающей выборкой и с помощью Pandas проведем первичный анализ данных.

Из библиотек нам понадобятся только NumPy и Pandas.

In [1]:
import numpy as np
import pandas as pd

Считываем данные из CSV-файла в объект pandas DataFrame.

In [2]:
df = pd.read_csv('mlbootcamp5_train.csv', sep=';', 
                 index_col='id')

Посмотрим на первые 5 записей.

In [5]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


**Сколько мужчин и женщин представлено в этом наборе данных? Не было дано расшифровки признака "пол" (какому полу соответствует 1, а какому – 2 в признаке gender) – это определите, посмотрев также на рост при разумном предположении, что в среднем мужчины выше (здесь и далее под средним понимается среднее арифметическое).**


* 45470 женщин и 24530 мужчин
* 45470 мужчин и 24530 женщин
* 45530 женщин и 24470 мужчин
* 45530 мужчин и 24470 женщин

In [11]:
df[df.gender == 1][['height']].describe()

,height
count,45530.000000
mean,161.355612
std,7.052726
min,55.000000
25%,157.000000
50%,161.000000
75%,166.000000
max,250.000000


In [12]:
df[df.gender == 2][['height']].describe()

,height
count,24470.000000
mean,169.947895
std,7.228895
min,65.000000
25%,165.000000
50%,170.000000
75%,175.000000
max,207.000000


In [53]:
g1 = df[df.gender == 1]['height']
g2 = df[df.gender == 2]['height']
print(g1.mean(), g2.mean())
print('Women: ',len(g1), 'Men: ',len(g2))

161.35561168460356 169.94789538210054
Women:  45530 Men:  24470


**Кто в среднем реже указывает, что употребляет алкоголь – мужчины или женщины?**


* мужчины
* женщины

In [55]:
print ('Women: ',round(df[df.gender == 1]['alco'].mean()*100,2), '%',
       'Men: ',round(df[df.gender == 2]['alco'].mean()*100,2), '%',)

Women:  2.55 % Men:  10.64 %


**Во сколько раз (округленно, round) процент курящих среди мужчин больше, чем процент курящих среди женщин (по крайней мере, по этим анкетным данным)?**

* 4
* 8
* 12
* 16

In [52]:
print (round(df[df.gender == 2]['smoke'].mean()/df[df.gender == 1]['smoke'].mean()))

12


**Вы наверняка заметили, что значения возраста какие-то странные. Догадайтесь, в чём здесь измеряется возраст, и ответьте, на сколько месяцев (примерно) отличаются медианные значения возраста курящих и некурящих.**


* 10
* 15
* 20
* 25

Days

In [67]:
round((df[df.smoke == 0]['age']/30).median()-(df[df.smoke == 1]['age']/30).median() )

20

**В статье на Википедии про сердечно-сосудистый риск показана шкала SCORE для расчёта риска смерти от сердечно-сосудистого заболевания в ближайшие 10 лет. Вот она:** <img src='https://habrastorage.org/webt/gg/38/vf/gg38vfl5ql8obqjmlsvt9es76p8.png' width=70%>

Давайте посмотрим на правый верхний прямоугольник, отображающий сегмент курящих мужчин в возрасте от 60 до 64 лет включительно. (Неочевидно, но тут для возраста и давления цифры означают верхнюю границу, и она не включается).

Видим 9-ку в левом нижнем углу этого прямоугольника и 47 – в правом верхнем. То есть если при этом систолическое (т.е. верхнее) артериальное давление – меньше 120 мм рт.ст., а уровень холестерина – 4 ммоль/л, то риск ССЗ оценивается примерно в 5 раз ниже, чем если бы давление лежало в интервале [160, 180), а холестерина было бы 8 ммоль/л.

Давайте посчитаем аналогичное значение, но на наших данных.

Уточнения:

* Создайте новый признак age_years – возраст в годах, округлив до целых (round). Для данного примера отберите курящих мужчин от 60 до 64 лет включительно
* Категории уровня холестрина на рисунке и в наших данных отличаются. Отображение значений на картинке в значения признака cholesterol следующее: 4 ммоль/л $\rightarrow$ 1, 5-7 ммоль/л $\rightarrow$ 2, 8 ммоль/л $\rightarrow$ 3.
* Интересуют 2 подвыборки курящих мужчин возраста от 60 до 64 лет включительно: первая с верхним артериальным давлением строго меньше 120 мм рт.ст. и концентрацией холестерина – 4 ммоль/л, а вторая – с верхним артериальным давлением от 160 (включительно) до 180 мм рт.ст. (не включительно) и концентрацией холестерина – 8 ммоль/л.

Во сколько раз (округленно, round) отличаются доли больных людей (согласно целевому признаку, cardio) в этих двух подвыборках? Посчитайте на наших данных.


* 2
* 3
* 4
* 5

In [3]:
df['age_years'] = round(df['age']/365)

In [22]:
subset1 = df[(df['age_years']>=60) & (df['age_years']<=64) & (df['ap_hi']<120) & (df['cholesterol']==1)]

In [24]:
subset2 = df[(df['age_years']>=60) & (df['age_years']<=64) & (df['ap_hi']>=160) & (df['ap_hi']<180) & (df['cholesterol']==3)]

In [50]:
ill_perc1 = subset1['cardio'].mean()
ill_perc2 = subset2['cardio'].mean()
round(ill_perc2/ill_perc1)

2

**Постройте новый признак – BMI (Body Mass Index). Для этого надо вес в килограммах поделить на квадрат роста в метрах. Нормальными считаются значения BMI от 18.5 до 25. Выберите верные утверждения.**


* Медианный BMI по выборке превышает норму
* У женщин в среднем BMI ниже, чем у мужчин
* У здоровых в среднем BMI выше, чем у больных
* В сегменте здоровых и непьющих мужчин в среднем BMI ближе к норме, чем в сегменте здоровых и непьющих женщин

In [34]:
df['BMI'] = df['weight']/((df['height']/100)**2)

In [37]:
df['BMI'].median()>25

True

In [48]:
BMI_w = df[df.gender == 1]['BMI']
BMI_m = df[df.gender == 2]['BMI']
print('Women: ', round(BMI_w.mean(),2), 'Men: ', round(BMI_m.mean(),2))
if (round(BMI_w.mean(),2)<round(BMI_m.mean(),2)):
    print('true')
else:
    print('false')

Women:  27.99 Men:  26.75
false


In [49]:
BMI_h = df[df.cardio == 0]['BMI']
BMI_i = df[df.cardio == 1]['BMI']
print('Health: ', round(BMI_h.mean(),2), 'Disease: ', round(BMI_i.mean(),2))
if (round(BMI_h.mean(),2)>round(BMI_i.mean(),2)):
    print('true')
else:
    print('false')

Health:  26.55 Disease:  28.57
false


In [58]:
BMI_hm = df[(df.cardio == 0) & (df.gender == 2) & (df.alco == 0)]['BMI']
BMI_hw = df[(df.cardio == 0) & (df.gender == 1) & (df.alco == 0)]['BMI']
print(BMI_hm.mean(), BMI_hw.mean())
if ((BMI_hm.mean()-25.5)<(BMI_hw.mean()-25.5)):
    print('true')
else:
    print('false')

25.872638075460173 26.845406594131518
true


Можно заметить, что данные не особо-то чистые, много в них всякой "грязи" и неточностей. Еще лучше мы это увидим, когда обсудим визуализацию данных.

Отфильтруйте следующие сегменты пациентов (считаем это ошибками в данных)

* указанное нижнее значение артериального давления строго выше верхнего
* рост строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили (используйте pd.Series.quantile, если не знаете, что это такое – прочитайте)
* вес строго меньше 2.5%-перцентили или строго больше 97.5%-перцентили
Это вовсе не вся чистка данных, которую можно было проделать, но пока остановимся на этом.

Сколько процентов данных (округленно, round) мы выбросили?


* 8
* 9
* 10
* 11

In [74]:
elim_data = df[(df['ap_lo'] > df['ap_hi']) | ((df['height']>df.quantile(.975).height) | (df['height']<df.quantile(.025).height)) 
   | ((df['weight']>df.quantile(.975).weight) | (df['weight']<df.quantile(.025).weight))].count().age

In [76]:
round((elim_data / df.count().age)*100)

10.0